In [40]:
import pandas as pd
import numpy as np 

In [41]:
def resample_and_interpolate(group):
    return group.reset_index(level=[1, 2]).resample('30T').asfreq().interpolate()

In [42]:
weather_df = pd.read_csv("/Users/florian/Documents/github/DP2/Energy_production_price_prediction/weather_data/DWD_ICON-EU.csv")
solar_total = pd.read_csv("/Users/florian/Documents/github/DP2/Energy_production_price_prediction/basic_files/solar_total_production.csv")
wind_total = pd.read_csv("/Users/florian/Documents/github/DP2/Energy_production_price_prediction/basic_files/wind_total_production.csv")
solar_total.generation_mw = solar_total.generation_mw * 0.5
wind_total.generation_mw = wind_total.generation_mw * 0.5 - wind_total.boa
weather_df.sort_values(by='ref_datetime', inplace=True)
weather_df = weather_df.groupby(["valid_datetime","latitude","longitude"]).last().reset_index()
weather_df.reset_index(inplace=True)

In [43]:
weather_df.valid_datetime = pd.to_datetime(weather_df.valid_datetime)
weather_df = weather_df.set_index(["valid_datetime","latitude","longitude"])
df_resampled = weather_df.groupby(['latitude', 'longitude'], group_keys=False).apply(resample_and_interpolate)
df_resampled = df_resampled.reset_index()

/var/folders/fm/5lbdpfj928d6sqhy_19cp6ww0000gn/T/ipykernel_60847/2834871732.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  return group.reset_index(level=[1, 2]).resample('30T').asfreq().interpolate()
/var/folders/fm/5lbdpfj928d6sqhy_19cp6ww0000gn/T/ipykernel_60847/2834871732.py:2: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  return group.reset_index(level=[1, 2]).resample('30T').asfreq().interpolate()
/var/folders/fm/5lbdpfj928d6sqhy_19cp6ww0000gn/T/ipykernel_60847/2834871732.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  return group.reset_index(level=[1, 2]).resample('30T').asfreq().interpolate()
/var/folders/fm/5lbdpfj928d6sqhy_19cp6ww0000gn/T/ipykernel_60847/2834871732.py:2: FutureWarning: DataFrame.interpolate with object dtype is depr

In [44]:
solar_total.timestamp_utc = pd.to_datetime(solar_total.timestamp_utc)
wind_total.timestamp_utc = pd.to_datetime(wind_total.timestamp_utc)
df_resampled.drop(columns=['index','ref_datetime'], inplace=True)
df_resampled_merged = pd.merge(df_resampled, solar_total, how='left', left_on='valid_datetime', right_on='timestamp_utc')
df_resampled_merged_solar = df_resampled_merged.loc[~(df_resampled_merged.latitude == 53.935) & ~(df_resampled_merged.longitude == 1.8645)]
df_resampled_merged_solar1 = df_resampled_merged_solar.groupby("valid_datetime").mean().reset_index()
distinct_lat_lon_pairs = df_resampled_merged_solar[['latitude', 'longitude']].drop_duplicates()

In [45]:
df_resampled_merged_wind = pd.merge(df_resampled, wind_total, how='inner', left_on='valid_datetime', right_on='timestamp_utc')
df_resampled_merged_wind = df_resampled_merged_wind.loc[(df_resampled_merged_wind.latitude == 53.935) & (df_resampled_merged_wind.longitude == 1.8645)]
df_resampled_merged_wind.drop_duplicates(inplace=True)
df_resampled_merged_wind


,valid_datetime,latitude,longitude,Temperature,WindSpeed,WindSpeed:100,WindDirection:100,CloudCover,RelativeHumidity,PressureReducedMSL,SolarDownwardRadiation,TotalPrecipitation,timestamp_utc,settlement_date,settlement_period,boa,generation_mw
1980,2024-10-07 00:00:00+00:00,53.935,1.8645,14.930556,8.763889,11.313333,159.952778,1.000000,93.232778,99385.651667,0.0,NaN,2024-10-07 00:00:00+00:00,2024-10-07,3,0.0,482.321
1982,2024-10-07 00:30:00+00:00,53.935,1.8645,14.891944,9.500139,12.232361,162.957083,1.000000,94.189028,99358.019583,0.0,NaN,2024-10-07 00:30:00+00:00,2024-10-07,4,0.0,541.471
1983,2024-10-07 01:00:00+00:00,53.935,1.8645,14.853333,10.236389,13.151389,165.961389,1.000000,95.145278,99330.387500,0.0,1.614722,2024-10-07 01:00:00+00:00,2024-10-07,5,0.0,569.911
1984,2024-10-07 01:30:00+00:00,53.935,1.8645,14.865694,9.685278,12.327083,184.986806,1.000000,94.389861,99286.175278,0.0,0.876389,2024-10-07 01:30:00+00:00,2024-10-07,6,0.0,568.161
1986,2024-10-07 02:00:00+00:00,53.935,1.8645,14.878056,9.134167,11.502778,204.012222,1.000000,93.634444,99241.963056,0.0,0.138056,2024-10-07 02:00:00+00:00,2024-10-07,7,0.0,568.211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2074,2024-10-08 19:30:00+00:00,53.935,1.8645,15.107778,4.958472,7.572917,178.350000,0.371111,88.645139,99239.545000,0.0,0.000417,2024-10-08 19:30:00+00:00,2024-10-08,42,0.0,395.712
2075,2024-10-08 20:00:00+00:00,53.935,1.8645,15.105000,5.028611,7.633056,178.156667,0.537778,88.688611,99231.603333,0.0,0.000000,2024-10-08 20:00:00+00:00,2024-10-08,43,0.0,291.852
2076,2024-10-08 20:30:00+00:00,53.935,1.8645,15.093194,5.382917,8.030694,174.153750,0.768889,88.485833,99228.149583,0.0,0.013333,2024-10-08 20:30:00+00:00,2024-10-08,44,0.0,282.192
2077,2024-10-08 21:00:00+00:00,53.935,1.8645,15.081389,5.737222,8.428333,170.150833,1.000000,88.283056,99224.695833,0.0,0.026667,2024-10-08 21:00:00+00:00,2024-10-08,45,0.0,255.792


In [46]:
def set_up_wind_features(df):
    R_d = 287.05  # Specific gas constant for dry air (J/(kg·K))
    R_v = 461.5   # Specific gas constant for water vapor (J/(kg·K))
    p = 101325    # Standard atmospheric pressure in Pa
    # Calculate saturation vapor pressure (using temperature in Celsius), Tetens formula
    df['Temperature_K'] = df['Temperature'] + 273.15
    e_s = 0.61078 * np.exp((17.27 * (df['Temperature'])) / (df['Temperature'] +237.3))
    # in pa
    e_s = 1000 * e_s
    # Calculate actual vapor pressure
    e = df['RelativeHumidity'] / 100 * e_s
    df['AirDensity'] = (p - e) / (R_d * df['Temperature_K']) + (e / (R_v * df['Temperature_K']))
    # Turbine stats
    rotor_diameter = 154  # in meters
    approximated_total_efficiency = 0.348
    limiter = 0.94
    minimum_wind_speed = 3  # in m/s
    maximum_wind_speed_for_power_curve = 12.5  # in m/s
    maximum_wind_speed_for_operation = 25  # in m/s
    rotor_area = np.pi * (rotor_diameter / 2) ** 2  # in m²
    # turbine requires 3m/s to start rotating
    const_internal_friction_coefficient = 0.5 * 1.240 * np.pi * 77**2 * 3**3 * approximated_total_efficiency * 174 / 1000000
    maximum_power_per_turbine = 7 # in MW
    # Same for full
    df['WindSpeed_full_avg'] = (df['WindSpeed'] + df['WindSpeed:100']) / 2
    df['WindPower_full'] = 0.5 * df['AirDensity'] * rotor_area * df['WindSpeed:100'] ** 3 * 174 / 1000000
    df['UsableWindPower_full'] = np.minimum(df['WindPower_full'], maximum_power_per_turbine * 174 * limiter / approximated_total_efficiency)
    df['PowerOutput_full'] = np.where((df['WindSpeed:100'] >= minimum_wind_speed) & (df['WindSpeed:100'] <= maximum_wind_speed_for_operation), df['UsableWindPower_full'] * approximated_total_efficiency - const_internal_friction_coefficient, 0) * 0.5

    # wind_df["Temperature_avg"] = (wind_df["Temperature"] + wind_df["Temperature:100"]) / 2
    # wind_df["RelativeHumidity_avg"] = (wind_df["RelativeHumidity"] + wind_df["RelativeHumidity:100"]) / 2
    df["Temperature_avg"] = df["Temperature"]
    df["RelativeHumidity_avg"] = df["RelativeHumidity"]   
    df["WindSpeed:100_dwd_lag1"] = df["WindSpeed:100"].shift(1)
    df["WindSpeed:100_dwd_lag2"] = df["WindSpeed:100"].shift(2)
    df["WindSpeed:100_dwd_lag3"] = df["WindSpeed:100"].shift(3)
    df["UsableWindPower_opt"] = df.UsableWindPower_full
    df["WindSpeed:100_dwd"] = df["WindSpeed:100"].shift(1)
    
    #df["cos_hour"] = np.cos(2 * np.pi * df["valid_datetime"].dt.hour / 24)
    #df["cos_day"] = np.cos(2 * np.pi * df["valid_datetime"].dt.dayofyear / 365)
    df["residual"] = df["generation_mw"] - df["PowerOutput_full"]
    #df["Wind_MWh_credit_Lag_48h"] = df["generation_mw"].shift(96)

    return df

In [47]:
df_resampled_merged_wind_2 = set_up_wind_features(df_resampled_merged_wind)
df_resampled_merged_wind_2.dropna(inplace=True)
# X_wind = df_resampled_merged_wind_2[['WindSpeed:100_dwd', 'Temperature_avg', 'RelativeHumidity_avg', 'AirDensity',"UsableWindPower_opt", 'WindSpeed:100_dwd_lag1', 'WindSpeed:100_dwd_lag2', 'WindSpeed:100_dwd_lag3']]
# y_wind = df_resampled_merged_wind_2['generation_mw']

train = df_resampled_merged_wind_2[['WindSpeed:100_dwd', 'Temperature_avg', 'RelativeHumidity_avg', 'AirDensity',"UsableWindPower_opt", 'WindSpeed:100_dwd_lag1', 'WindSpeed:100_dwd_lag2', 'WindSpeed:100_dwd_lag3', 'PowerOutput_full', 'residual', 'generation_mw']]

In [48]:
df_test = train.copy()

df_test.rename(columns={"generation_mw": "Wind_MWh_credit", "PowerOutput_full": "PowerOutput_opt"}, inplace=True)

In [49]:
df_test.columns = [col.replace(":", "_") for col in df_test.columns]  
df_test["WindSpeed_100_dwd_std"] = abs(df_test["WindSpeed_100_dwd"] - df_test["WindSpeed_100_dwd"].mean())
#df_test["WindSpeed_100_dwd_2"] = df_test["WindSpeed_100_dwd"] ** 2
#df_test["WindSpeed_100_dwd_3"] = df_test["WindSpeed_100_dwd"] ** 3

In [50]:
df_test

,WindSpeed_100_dwd,Temperature_avg,RelativeHumidity_avg,AirDensity,UsableWindPower_opt,WindSpeed_100_dwd_lag1,WindSpeed_100_dwd_lag2,WindSpeed_100_dwd_lag3,PowerOutput_opt,residual,Wind_MWh_credit,WindSpeed_100_dwd_std
1984,13.151389,14.865694,94.389861,1.218287,3290.000000,13.151389,12.232361,11.313333,563.019711,5.141289,568.161,4.444792
1986,12.327083,14.878056,93.634444,1.218288,3004.751994,12.327083,13.151389,12.232361,513.386558,54.824442,568.211,3.620486
1987,11.502778,14.742222,93.060556,1.218970,2951.020062,11.502778,12.327083,13.151389,504.037202,62.753798,566.791,2.796181
1989,11.431667,14.606389,92.486667,1.219651,2897.910196,11.431667,11.502778,12.327083,494.796085,25.374915,520.171,2.725069
1990,11.360556,14.319583,91.888333,1.221042,3290.000000,11.360556,11.431667,11.502778,563.019711,-75.629711,487.390,2.653958
...,...,...,...,...,...,...,...,...,...,...,...,...
2074,7.512778,15.107778,88.645139,1.217600,856.929085,7.512778,7.458611,7.404444,139.665372,256.046628,395.712,1.193819
2075,7.572917,15.105000,88.688611,1.217610,877.513956,7.572917,7.512778,7.458611,143.247140,148.604860,291.852,1.133681
2076,7.633056,15.093194,88.485833,1.217681,1021.982377,7.633056,7.572917,7.512778,168.384645,113.807355,282.192,1.073542
2077,8.030694,15.081389,88.283056,1.217752,1181.502216,8.030694,7.633056,7.572917,196.141097,59.650903,255.792,0.675903


In [51]:
df_test.to_csv("/Users/florian/Documents/github/DP2/Energy_production_price_prediction/Generation_forecast/Solar_forecast/data/wind_test.csv", index=False)    